In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import imodelsx.process_results
from collections import defaultdict
import numpy as np
import viz
import transformers
import sys
import tprompt.utils
from os.path import join
import datasets
from typing import Dict, List
from sklearn.tree import plot_tree
import imodelsx.util
import imodelsx.metrics
import numpy as np
import tprompt.utils
from scipy.special import softmax
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import torch.cuda
import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.base import BaseEstimator, ClassifierMixin
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
import torch
sys.path.append('../experiments/')
import os
import pandas as pd
import numpy as np
import transformers
import sys
from os.path import join
import datasets
from typing import Dict, List
from dict_hash import sha256
import joblib
import random
import numpy as np
import sklearn.tree
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import OneHotEncoder

import imodelsx.treeprompt.stump
from ensembling import PromptHooker, modify_activations

In [ ]:
# hyperparams
checkpoint = 'gpt2'
prompts = [
    " This movie is",
    " Positive or Negative? The movie was",
    " The sentiment of the movie was",
    " The plot of the movie was really",
    " The acting in the movie was",
]
# note also requires specifying the layer for the hook below


tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).eval()

In [ ]:
OUTPUTS_ALL = {}
PROMPT_NUM_GLOBAL = 0


def store_activations(module, inputs, outputs):
    global OUTPUTS_ALL
    global PROMPT_NUM_GLOBAL

    # Define the hook function
    # print(module)
    OUTPUTS_ALL[PROMPT_NUM_GLOBAL] = outputs
    # print(OUTPUTS_ALL)
    PROMPT_NUM_GLOBAL += 1
    return outputs * 3  # * 0.01


hook = model.transformer.drop.register_forward_hook(store_activations)
for i, prompt in enumerate(prompts):
    inputs = tokenizer(prompt, return_tensors="pt")
    # hook = model.transformer.h[3].register_forward_hook(change_activations)
    logits_modified = model(**inputs).logits

hook.remove()
assert len(OUTPUTS_ALL) == len(prompts)

# most_probable_tokens = torch.topk(logits_modified, k=10, dim=-1)
# print('\n'.join([tokenizer.decode(x)
#   for x in most_probable_tokens.indices[0, -1]]))
# logits_orig = model(**inputs).logits

In [12]:
for i in range(len(prompts)):
    print(i, prompts[i], OUTPUTS_ALL[i].shape)

0  This movie is torch.Size([1, 3, 768])
1  Positive or Negative? The movie was torch.Size([1, 7, 768])
2  The sentiment of the movie was torch.Size([1, 6, 768])
3  The plot of the movie was really torch.Size([1, 7, 768])
4  The acting in the movie was torch.Size([1, 6, 768])


# Evaluate accuracies

In [ ]:
verbalizer = {0: " Negative.", 1: " Positive."}

In [ ]:
dset_train = datasets.load_dataset('rotten_tomatoes')['train']
dset_train = dset_train.select(np.random.choice(
    len(dset_train), size=100, replace=False))
dset_val = datasets.load_dataset('rotten_tomatoes')['validation']
dset_val = dset_val.select(np.random.choice(
    len(dset_val), size=100, replace=False))

In [ ]:
m = PromptHooker(
    checkpoint='gpt2',
    prompts=prompts,
    verbalizer=verbalizer,
    cache_prompt_features_dir=None,
    random_state=42,
    hook_weights=[1, 1, 1, 1, 1],
)

In [ ]:
m.fit(dset_train["text"], dset_train["label"])

# compute accuracy
print('Predicting on validation set...')

# compare to accuracy for individual prompts
for i, prompt in enumerate(prompts):
    print(i, prompt, '->', m.prompt_accs_[i])